In [ ]:
#!/usr/bin/env python3
"""
Multi-image → video через Kling (модель kling-video/v1.6/standard/multi-image-to-video, формат по AIMLAPI).

Что делает скрипт:
1. Берёт 2–4 URL изображений (аргумент --image-urls).
2. Шлёт POST на создание задачи:
   POST https://api.aimlapi.com/v2/generate/video/kling/generation
   {
       "model": "kling-video/v1.6/standard/multi-image-to-video",
       "image_list": [...],
       "prompt": "...",
       "negative_prompt": "...",
       "duration": 5|10,
       "aspect_ratio": "16:9"|"9:16"|"1:1"
   }
   (см. схему в доке AIMLAPI).  # https://docs.aimlapi.com/.../v1.6-standard/multi-image-to-video
3. Забирает результат:
   GET  https://api.aimlapi.com/v2/generate/video/kling/generation?generation_id=...
4. Печатает video.url и при необходимости скачивает его в mp4.

Адаптация под native Kling:
- BASE_URL, CREATE_URL, STATUS_URL, AUTH_HEADER нужно заменить на свои из доки Kling.
- Поля body (image_list, prompt, duration, aspect_ratio, negative_prompt) совпадают с "официальным" форматом MultiImage2Video.
"""

import argparse
import os
import sys
import time
from pathlib import Path
from typing import List, Optional

import requests


# =========================
#  Конфиг
# =========================

# Для AIMLAPI (агрегатор Kling)
AIMLAPI_BASE_URL = "https://api.aimlapi.com"
AIMLAPI_CREATE_URL = f"{AIMLAPI_BASE_URL}/v2/generate/video/kling/generation"
AIMLAPI_STATUS_URL = f"{AIMLAPI_BASE_URL}/v2/generate/video/kling/generation"

# Модель multi-image-to-video из доки AIMLAPI:
# model: "kling-video/v1.6/standard/multi-image-to-video"
# type:  "multi-image-to-video" (опционально).  # https://docs.aimlapi.com/.../v1.6-standard/multi-image-to-video
DEFAULT_MODEL = "kling-video/v1.6/standard/multi-image-to-video"


class KlingApiError(Exception):
    """Своя ошибка для удобства."""


def get_env(name: str) -> str:
    """Берём переменную окружения или падаем с понятной ошибкой."""
    value = os.environ.get(name)
    if not value:
        raise KlingApiError(f"Переменная окружения {name} не задана")
    return value


def create_multi_image_task(
    api_key: str,
    image_urls: List[str],
    prompt: Optional[str],
    negative_prompt: Optional[str],
    duration: int,
    aspect_ratio: str,
    model: str = DEFAULT_MODEL,
    external_task_id: Optional[str] = None,
) -> str:
    """
    Создаёт задачу multi-image → video.

    Для AIMLAPI формат тела (по доке):
        {
          "model": "kling-video/v1.6/standard/multi-image-to-video",
          "type": "multi-image-to-video",          # опционально
          "image_list": ["url1", "url2", ...],     # 2–4 URL
          "prompt": "...",                         # опционально
          "negative_prompt": "...",                # опционально
          "duration": 5|10,                        # опционально
          "aspect_ratio": "16:9"|"9:16"|"1:1",     # опционально
          "external_task_id": "..."                # опционально
        }
    Ответ:
        { "id": "...", "status": "...", ... }

    Возвращает generation_id (id задачи).
    """
    if len(image_urls) < 2:
        raise KlingApiError("Нужно минимум 2 изображения для multi-image-to-video")
    if len(image_urls) > 4:
        raise KlingApiError("Максимум 4 изображения (ограничение Kling 1.6 multi-image)")

    headers = {
        # Для AIMLAPI:
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    body: dict = {
        "model": model,
        "image_list": image_urls,
        "duration": duration,
        "aspect_ratio": aspect_ratio,
    }

    # Тип задачи (в AIMLAPI помечен как enum "multi-image-to-video", но можно не указывать)
    body["type"] = "multi-image-to-video"

    if prompt:
        body["prompt"] = prompt
    if negative_prompt:
        body["negative_prompt"] = negative_prompt
    if external_task_id:
        body["external_task_id"] = external_task_id

    resp = requests.post(AIMLAPI_CREATE_URL, headers=headers, json=body, timeout=60)
    if resp.status_code != 200:
        raise KlingApiError(
            f"Ошибка при создании задачи: HTTP {resp.status_code} {resp.text}"
        )

    data = resp.json()

    # По доке AIMLAPI пример успешного ответа:
    # {
    #   "id": "60ac7c34-3224-4b14-8e7d-0aa0db708325",
    #   "status": "completed",
    #   "video": { "url": "...", "duration": 8 },
    #   ...
    # }
    generation_id = data.get("id")
    if not generation_id:
        raise KlingApiError(f"В ответе нет поля 'id': {data}")

    status = data.get("status")
    print(f"[create] generation_id={generation_id}, status={status}")
    return generation_id


def poll_video_result(
    api_key: str,
    generation_id: str,
    poll_interval: int = 5,
    timeout_sec: int = 600,
) -> str:
    """
    Пуллит результат до статуса "completed" и возвращает video.url.

    GET /v2/generate/video/kling/generation?generation_id=...
    Ответ (по доке):
    {
      "id": "...",
      "status": "completed",
      "video": {
        "url": "https://....mp4",
        "duration": 8
      },
      ...
    }
    """
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
    }

    deadline = time.time() + timeout_sec
    while True:
        if time.time() > deadline:
            raise KlingApiError(
                f"Таймаут ожидания {timeout_sec} с для задачи {generation_id}"
            )

        params = {"generation_id": generation_id}
        resp = requests.get(
            AIMLAPI_STATUS_URL, headers=headers, params=params, timeout=30
        )
        if resp.status_code != 200:
            raise KlingApiError(
                f"Ошибка при запросе статуса: HTTP {resp.status_code} {resp.text}"
            )

        data = resp.json()
        status = data.get("status")
        print(f"[poll] status={status}")

        if status == "completed":
            video = data.get("video") or {}
            url = video.get("url")
            if not url:
                raise KlingApiError(
                    f"status=completed, но нет video.url в ответе: {data}"
                )
            print(f"[done] video_url={url}")
            return url

        if status in {"failed", "error"}:
            raise KlingApiError(f"Задача {generation_id} завершилась со статусом {status}")

        time.sleep(poll_interval)


def download_video(url: str, output_path: Path) -> None:
    """Скачиваем mp4 по ссылке."""
    print(f"[download] {url} -> {output_path}")
    with requests.get(url, stream=True, timeout=600) as r:
        r.raise_for_status()
        with output_path.open("wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    print("[download] done")


def parse_args(argv: Optional[List[str]] = None) -> argparse.Namespace:
    """Парсер аргументов CLI."""
    parser = argparse.ArgumentParser(
        description="Multi-image → video через Kling (формат AIMLAPI multi-image-to-video)"
    )

    parser.add_argument(
        "--image-urls",
        nargs="+",
        required=True,
        help="Список URL изображений (2–4 шт., должны быть доступны по HTTP/HTTPS)",
    )
    parser.add_argument(
        "--prompt",
        default=None,
        help="Позитивный текстовый промпт (опционально)",
    )
    parser.add_argument(
        "--negative-prompt",
        default=None,
        help="Негативный промпт (чего НЕ должно быть на видео, опционально)",
    )
    parser.add_argument(
        "--duration",
        type=int,
        default=5,
        choices=[5, 10],
        help="Длительность видео (секунды, 5 или 10)",
    )
    parser.add_argument(
        "--aspect-ratio",
        default="16:9",
        choices=["16:9", "9:16", "1:1"],
        help="Соотношение сторон",
    )
    parser.add_argument(
        "--model",
        default=DEFAULT_MODEL,
        help=f"Имя модели (по умолчанию {DEFAULT_MODEL})",
    )
    parser.add_argument(
        "--external-task-id",
        default=None,
        help="Произвольный external_task_id (опционально, для трекинга с вашей стороны)",
    )
    parser.add_argument(
        "--output",
        default=None,
        help="Путь для сохранения mp4 (если не указан — просто выводим URL)",
    )

    return parser.parse_args(argv)


def main(argv: Optional[List[str]] = None) -> int:
    try:
        args = parse_args(argv)

        # Для AIMLAPI нужно задать AIMLAPI_API_KEY
        api_key = get_env("AIMLAPI_API_KEY")

        # 1. Создаём задачу
        generation_id = create_multi_image_task(
            api_key=api_key,
            image_urls=args.image_urls,
            prompt=args.prompt,
            negative_prompt=args.negative_prompt,
            duration=args.duration,
            aspect_ratio=args.aspect_ratio,
            model=args.model,
            external_task_id=args.external_task_id,
        )

        # 2. Ждём готовности, получаем URL видео
        video_url = poll_video_result(
            api_key=api_key,
            generation_id=generation_id,
        )

        print("\nГотовое видео:")
        print(video_url)

        # 3. При необходимости — качаем
        if args.output:
            output_path = Path(args.output)
            download_video(video_url, output_path)

        return 0

    except KlingApiError as e:
        print(f"Ошибка: {e}", file=sys.stderr)
        return 1
    except KeyboardInterrupt:
        print("\nПрервано пользователем", file=sys.stderr)
        return 130


if __name__ == "__main__":
    raise SystemExit(main())